In [6]:
import numpy as np
import matplotlib.pyplot as plt

def simulate_variance_regimes(
    n_steps=500,
    p_switch=0.01,
    sigma0=0.25,
    seed=41,
):
    """
    Simulate a stepwise constant variance process with geometric regime durations.

    Parameters
    ----------
    n_steps : int
        Total number of time steps.
    p_switch : float
        Per-step probability that the current regime ends
        and a new variance level is drawn.
    sigma0 : float
        Baseline variance scale used in the Gamma(3/2, rate=sigma0^2) model.
    seed : int
        Random seed for reproducibility.

    Returns
    -------
    V : ndarray of shape (n_steps,)
        Variance level V_t at each time step.
    regime_boundaries : list of ints
        Indices where regimes start (0 = first regime start).
    """
    rng = np.random.default_rng(seed)

    # Gamma parameters for precision tau
    alpha = 1.5
    beta = sigma0**2  # rate

    V = np.zeros(n_steps)
    regime_boundaries = [0]

    # draw initial regime variance
    tau = rng.gamma(shape=alpha, scale=1.0 / beta)
    V_current = 1.0 / tau

    for t in range(n_steps):
        V[t] = V_current

        # decide if regime switches at the *next* step
        if rng.random() < p_switch and t < n_steps - 1:
            regime_boundaries.append(t + 1)
            tau = rng.gamma(shape=alpha, scale=1.0 / beta)
            V_current = 1.0 / tau

    return V, regime_boundaries


def plot_variance_regimes(
    V,
    regime_boundaries,
    filename="variance_regimes.png",
):
    """
    Plot stepwise variance levels with regime switches and save to disk.
    """
    fig, ax = plt.subplots(figsize=(8, 3))

    t = np.arange(len(V))
    ax.step(t, V, where="post", linewidth=2)

    # Mark regime switches with vertical dashed lines
    for rb in regime_boundaries[1:]:
        ax.axvline(rb, linestyle="--", linewidth=1, alpha=0.4)

    # Labels
    ax.set_xlabel("Time step $t$")
    ax.set_ylabel("Variance level $V_t$")

    # Remove top and right spines
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    # Simple annotations for the first couple of regimes (if they exist)
    if len(regime_boundaries) >= 1:
        ax.annotate(
            "Regime 1",
            xy=(regime_boundaries[0], V[regime_boundaries[0]]),
            xytext=(10, 10),
            textcoords="offset points",
            arrowprops=dict(arrowstyle="->", lw=1),
        )
    if len(regime_boundaries) >= 2:
        idx = regime_boundaries[1]
        ax.annotate(
            "Regime 2",
            xy=(idx, V[idx]),
            xytext=(10, -30),
            textcoords="offset points",
            arrowprops=dict(arrowstyle="->", lw=1),
        )

    ax.set_title("Example of stepwise variance with geometric regime durations")

    fig.tight_layout()
    fig.savefig(filename, dpi=300, bbox_inches="tight")
    plt.close(fig)


if __name__ == "__main__":
    V, regime_boundaries = simulate_variance_regimes(
        n_steps=500,
        p_switch=0.01,
        sigma0=0.25,
        seed=2,
    )
    plot_variance_regimes(V, regime_boundaries, filename="variance_regimes.png")
    print("Saved variance_regimes.png")


Saved variance_regimes.png
